In [1]:
import pandas as pd
import csv


In [3]:
w1data = pd.read_csv("nfl-big-data-bowl-2023/week1.csv")
w1data.head()

,gameId,playId,nflId,frameId,time,jerseyNumber,team,playDirection,x,y,s,a,dis,o,dir,event
0,2021090900,97,25511.0,1,2021-09-10T00:26:31.100,12.0,TB,right,37.77,24.22,0.29,0.30,0.03,165.16,84.99,None
1,2021090900,97,25511.0,2,2021-09-10T00:26:31.200,12.0,TB,right,37.78,24.22,0.23,0.11,0.02,164.33,92.87,None
2,2021090900,97,25511.0,3,2021-09-10T00:26:31.300,12.0,TB,right,37.78,24.24,0.16,0.10,0.01,160.24,68.55,None
3,2021090900,97,25511.0,4,2021-09-10T00:26:31.400,12.0,TB,right,37.73,24.25,0.15,0.24,0.06,152.13,296.85,None
4,2021090900,97,25511.0,5,2021-09-10T00:26:31.500,12.0,TB,right,37.69,24.26,0.25,0.18,0.04,148.33,287.55,None


In [17]:
def collect_game_info(weekdf, playId, saved=False):
    
    #slice df to single play
    playdf = weekdf.loc[weekdf["playId"] == playId]
    #reorder playdf so that they are in the right frame order
    playdf = playdf.sort_values(by = "frameId")
    if saved:
        playdf.to_csv(f'Play{playId}Frames.csv')
    return playdf

In [18]:
collect_game_info(w1data, 97,True)

,gameId,playId,nflId,frameId,time,jerseyNumber,team,playDirection,x,y,s,a,dis,o,dir,event
0,2021090900,97,25511.0,1,2021-09-10T00:26:31.100,12.0,TB,right,37.77,24.22,0.29,0.30,0.03,165.16,84.99,None
817,2021090900,97,52459.0,1,2021-09-10T00:26:31.100,7.0,DAL,right,43.50,16.27,0.80,0.57,0.08,311.97,202.95,None
172,2021090900,97,40151.0,1,2021-09-10T00:26:31.100,66.0,TB,right,42.18,23.97,0.00,0.00,0.01,45.36,206.92,None
129,2021090900,97,39985.0,1,2021-09-10T00:26:31.100,25.0,TB,right,37.54,22.26,0.11,0.07,0.02,91.78,132.28,None
645,2021090900,97,44962.0,1,2021-09-10T00:26:31.100,18.0,DAL,right,58.60,27.33,0.55,0.87,0.06,299.23,140.01,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
945,2021090900,97,53504.0,43,2021-09-10T00:26:35.300,97.0,DAL,right,33.17,19.71,3.81,3.09,0.41,290.21,256.40,None
42,2021090900,97,25511.0,43,2021-09-10T00:26:35.300,12.0,TB,right,32.07,19.54,3.79,2.32,0.38,62.63,217.30,None
730,2021090900,97,46163.0,43,2021-09-10T00:26:35.300,65.0,TB,right,35.00,26.16,1.65,2.76,0.16,234.64,272.80,None
773,2021090900,97,47996.0,43,2021-09-10T00:26:35.300,6.0,DAL,right,46.47,17.04,7.02,0.74,0.70,83.63,186.98,None
